# Travaux Pratiques 1

Le but de ce TP est de manipuler les différentes notions abordées dans la vidéo du module 1.

Il y a trois objectifs principaux dans ce TP :
1. étudier l'impact de la suppression des valeurs manquantes sur les résultats (exercice 1),
2. illustrer le concept d'ignorabilité du mécanisme de données manquantes (exercice 2),
3. savoir générer des valeurs manquantes (exercices 3 et 4 principalement).

### Définition de l'*amputation*

Un jeu de données est dit *amputé* s'il contient des valeurs manquantes qui ont été générées. Le processus d'*amputation* est le fait de passer d'un jeu de données complet à un jeu de données incomplet ou d'un jeu de données incomplet à un jeu de données incomplet avec davantage de valeurs manquantes. En fait, on parle d'amputation quand on introduit des valeurs manquantes dans le jeu de données initial. C'est exactement le contraire de l'*imputation*.

C'est très important dans le traitement des valeurs manquantes : cela permet de pouvoir tester de nouveaux algorithmes ou de comparer des méthodes en ayant accès à un score de référence et en ayant accès aux valeurs observées, requises pour le calcul de certains scores.

# Importation de librairies

In [ ]:
### Classical libraries
import numpy as np
import pandas as pd
from scipy import optimize

### Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors

### Real datasets
from sklearn.datasets import load_breast_cancer

### Libraries specific to handle missing values
import pyampute
import missingno

# Exercice 1 : suppression des individus incomplets

Considérons un jeu de données composé de $n$ échantillons $(X_{1.},\dots,X_{n.})$ i.i.d. Gaussiens, tels quel $X_{i.} \sim N({\mu},{\Sigma}),$ avec $\mu \in \mathbb{R}^d$ and ${\Sigma} \in \mathbb{R}^{d\times d}$.

Le but est d'étudier empiriquement l'impact de la suppression des individus incomplets. Un individu (ou une ligne, ou un échantillon) est dit incomplet s'il contient au moins une valeur manquante.

Reprenons l'exemple de Zhu et al. (2022) donné dans la vidéo du module 1. On considère un jeu de données avec $d$ variables et un pourcentage de valeurs manquantes égal à 1\%. En petite dimension ($d=1$), si les individus incomplets sont supprimés, il reste 95\% d'individus complets. En grande dimension ($d=300$), il ne reste que 5\% d'individus complets.

Dans cet exercice, vous allez illustrer cet exemple et étudier l'impact de la suppression des individus incomplets sur les biais de la moyenne empirique comme estimateur de l'espérance.

In [ ]:
n = 1000
d = 5
Mu = np.repeat(0, d)
Sigma = 0.5 * (np.ones((d,d)) + np.eye(d))

xfull = np.random.multivariate_normal(Mu, Sigma, size=n) # complete dataset

In [ ]:
pd.DataFrame(xfull).head()

Vous allez d'abord générer des valeurs manquantes de type Missing Completely At Random (MCAR) : le manque des données ne dépend pas des valeurs des données elles-mêmes.

## Question 1 : génération de valeurs manquantes MCAR

Pour générer des valeurs manquantes MCAR, est-ce que la solution suivante vous semble satisfaisante ? Dans le cas contraire, proposez une meilleure stratégie.

In [ ]:
p = 0.4
xmiss = np.copy(xfull)
for j in range(d):
    miss_id = np.random.choice(n, np.floor(n*p).astype(int), replace=False)
    xmiss[miss_id, j] = np.nan
M = np.isnan(xmiss) # mask: matrix indicating where the missing values are in the data

In [ ]:
print("Le pourcentage total de NAs est:", np.sum(M) / (n*d))

### Solution

Cette méthode ne considère pas la nature stochastique du masque $M$, indiquant où sont les valeurs manquantes. Cela revient à considérer que $p$ est un pourcentage (exact) de valeurs manquantes. C'est d'ailleurs pour cela qu'un pourcentage de valeurs manquantes égal à 40% exactement est obtenu (relancez le code pour observer cela !). Il vaut mieux tirer le masque $M$ selon une loi de Bernoulli de paramètre $p$, qui est une probabilité d'être manquant.

In [ ]:
xmiss = np.copy(xfull)
for j in range(d):
  miss_id = np.random.uniform(0, 1, size=n) < p
  xmiss[miss_id, j] = np.nan
M = np.isnan(xmiss)

In [ ]:
pd.DataFrame(xmiss).head()

In [ ]:
print("Le pourcentage total de NAs est:", np.sum(M) / (n*d))

## Question 2: calcul du biais de la moyenne empirique

Le jeu de données `xmiss` contient à présent des valeurs manquantes ; c'est le jeu de donnés amputé. Vous allez calculer la moyenne empirique des variables en supprimant les individus incomplets et la comparer à la moyenne empirique si tous les individus étaient observés.

Proposez un code pour calculer les biais empiriques de la moyenne dans ces deux cas (par variable dans un premier temps).

In [ ]:
x_cc= pd.DataFrame(xmiss).dropna()
x_cc.shape

### Solution

In [ ]:
empirical_mean = np.mean(xfull, axis=0)
empirical_mean_na = np.mean(x_cc, axis=0)

bias = empirical_mean - Mu
bias_na = empirical_mean_na - Mu

print("Biais sans NA:", [f"{x:.3f}" for x in bias])
print("Biais avec NA:", [f"{x:.3f}" for x in bias_na])

On calcule la norme 2 du vecteur de biais pour obtenir une seule valeur.

In [ ]:
norm2_bias = (bias ** 2).sum()
norm2_bias_na = (bias_na ** 2).sum()

print("Norme 2 du biais sans NA:", f"{norm2_bias:.3f}")
print("Norme 2 du biais avec NA:", f"{norm2_bias_na:.3f}")

## Question 3 : comparaison sur plusieurs simulations

L'objectif est maintenant de reproduire l'expérience pour plusieurs valeurs de $d$ (nombres de variables du jeu de données) et de $p$ (probabilité d'être manquant). Pour avoir l'ordre de grandeur du biais, on veut reproduire l'expérience plusieurs fois dans chaque cas.

Comment compléter la fonction `compute_bias` pour qu'elle renvoie les biais sur plusieurs simulations ? La fonction prend en entrée le nombre de simulations `n_sim`, la probabilité pour une valeur du jeu de données d'être manquante `p`, le jeu de données complet `xfull` et la moyenne théorique `Mu`.

In [ ]:
def compute_bias(n_sim, p, xfull, Mu):
    vec_norm2_bias = []
    vec_norm2_bias_na = []

    d = xfull.shape[1]

    for it in range(n_sim):

        ### Generate missing values ###

        ### TO COMPLETE ###

        vec_norm2_bias.append(norm2_bias)
        vec_norm2_bias_na.append(norm2_bias_na)

    return(vec_norm2_bias, vec_norm2_bias_na)

On peut tester la fonction avec les arguments suivants: `n_sim`=10, `p`=10%. Puis, on applique la fonction pour avoir les valeurs des biais sur plusieurs possibilités pour le nombre de variables et plusieurs valeurs de probabilité d'être manquant.

### Remarque: stochasticité avec valeurs manquantes

Dans le cas complet, quand on veut faire plusieurs simulations dans un jeu de données synthétiques, ce que le l'on peut faire, c'est que l'on génère le jeu de données selon la même loi avec les paramètres théoriques, ici $({\Sigma},{\mu})$, plusieurs fois. Ici, la stochasticité va venir de la génération des valeurs manquantes. On considère le jeu de données complet `xfull` fixe, et on génère les valeurs manquantes plusieurs fois, on obtient donc des jeux de données amputés `xmiss` différents.

### Solution

In [ ]:
def compute_bias(n_sim, p, xfull, Mu):
    vec_norm2_bias_na = []

    n = xfull.shape[0]
    d = xfull.shape[1]

    empirical_mean = np.mean(xfull,axis=0)
    bias = empirical_mean - Mu
    norm2_bias = np.sqrt((bias ** 2).sum())

    for it in range(n_sim):

        ### Generate missing values ###
        xmiss = np.copy(xfull)
        for j in range(d):
          miss_id = np.random.uniform(0, 1, size=np.floor(n).astype(int)) < p
          xmiss[miss_id, j] = np.nan

        x_cc = pd.DataFrame(xmiss).dropna()

        if x_cc.shape[0] == 0:
          vec_norm2_bias_na.append(np.nan)

        empirical_mean_na = np.mean(x_cc, axis=0)

        bias_na = empirical_mean_na - Mu

        norm2_bias_na = (bias_na ** 2).sum()

        vec_norm2_bias_na.append(norm2_bias_na)

    return(norm2_bias, vec_norm2_bias_na)

In [ ]:
n_sim = 10
p = 0.1
norm2_bias, vec_norm2_bias_na = compute_bias(n_sim=n_sim, p=0.1, xfull=xfull, Mu=Mu)

In [ ]:
print("Biais empirique:", f"{norm2_bias:.3f}")
print("Moyenne des biais empiriques avec NA sur", f"{n_sim}", "simulations:", f"{np.mean(vec_norm2_bias_na):.3f}")

In [ ]:
d_list = [5, 10, 100]
p_list = [0.01, 0.05, 0.1, 0.5]

vec_norm2_bias = np.zeros(len(d_list))
mat_norm2_bias_na = np.zeros((len(d_list), len(p_list)))

for pos_d, d in enumerate(d_list):

    ### Complete dataset
    Mu = np.repeat(0, d)
    Sigma = 0.5 * (np.ones((d,d)) + np.eye(d))
    xfull = np.random.multivariate_normal(Mu, Sigma, size=n)

    for pos_perc, p in enumerate(p_list):
        norm2_bias, vec_norm2_bias_na = compute_bias(n_sim=10, p=p, xfull=xfull, Mu=Mu)
        mat_norm2_bias_na[pos_d, pos_perc] = round(np.mean(vec_norm2_bias_na), 3)

    vec_norm2_bias[pos_d] = round(norm2_bias, 3)

In [ ]:
results = pd.DataFrame(vec_norm2_bias, index=[f"d={d}" for d in d_list], columns=['Without NA'])
results_na = pd.DataFrame(mat_norm2_bias_na, index=[f"d={d}" for d in d_list], columns=[f"p={p}" for p in p_list])

results.join(results_na)

## Question 4: interprétation des résultats

Interprétez les résultats obtenus en question 3.

### Solution

Le biais empirique est du même ordre de grandeur pour $d=5$ variables et une probabilité d'être manquant de $p=1\%$ à $p=10\%$ ou pour $d=10$ variables et $p=1\%$. Sinon, pour les autres cas, le bais empirique de la moyenne est nettement plus élevé en présence de valeurs manquantes.

Il y a des `NA` dans le tableau de résultats lorsqu'il y a au moins une simulation où il n'y a aucun individu complet. Dans la cellule de code suivante, on définit la fonction `compute_number_complete_individuals` pour afficher le nombre d'individus complets.

In [ ]:
def compute_number_complete_individuals(n_sim,p,xfull):
    vec_complete_individuals = []

    n = xfull.shape[0]
    d = xfull.shape[1]

    for it in range(n_sim):
        np.random.seed(it)

        ### Generation of missing values
        xmiss = np.copy(xfull)
        for j in range(d):
          miss_id = np.random.uniform(0, 1, size=np.floor(n).astype(int)) < p
          xmiss[miss_id, j] = np.nan

        x_cc = pd.DataFrame(xmiss).dropna()

        number_complete_individuals = x_cc.shape[0]

        vec_complete_individuals.append(number_complete_individuals)

    return(vec_complete_individuals)

In [ ]:
d_list = [5, 10, 100]
p_list = [0.01, 0.05, 0.1, 0.5]

mat_complete_individuals = np.zeros((len(d_list), len(p_list)))

for pos_d, d in enumerate(d_list):

    ### Jeu de données complet
    Mu = np.repeat(0, d)
    Sigma = 0.5 * (np.ones((d, d)) + np.eye(d))
    xfull = np.random.multivariate_normal(Mu, Sigma, size=n)

    for pos_perc, p in enumerate(p_list):
        vec_complete_individuals = compute_number_complete_individuals(n_sim=10, p=p, xfull=xfull)
        mat_complete_individuals[pos_d, pos_perc] = round(np.mean(vec_complete_individuals) / (n*d)*100, 2)

Dans ce tableau, on affiche le pourcentage d'individus complets dans chaque cas.

In [ ]:
percentage_complete_individuals = pd.DataFrame(mat_complete_individuals, index=[f"d={d}" for d in d_list], columns=[f"p={p}" for p in p_list])

percentage_complete_individuals

# Exercice 2 : ignorabilité du mécanisme de données manquantes

Dans cet exercice, vous allez illustrer le concept d'ignorabilité du mécanisme.

Dans la vidéo du MOOC, vous avez vu que le mécanisme de données manquantes est ignorable s'il est MCAR ou MAR,  et non-ignorable dans le cas MNAR. Rappelons que le mécanisme est Missing At Random (MAR) si le manque des données ne dépend que des valeurs des données observées, et Missing Not At Random (MNAR) si le manque des données peut dépendre de toutes les valeurs des données, dont celles manquantes.

Considérons des données Gaussiennes bivariées, le même jeu de données que dans l'exercice 1 avec $d=2$ variables.

In [ ]:
n = 1000
d = 2
Mu = np.repeat(0, d)
Sigma = 0.5 * (np.ones((d, d)) + np.eye(d))

xfull = np.random.multivariate_normal(Mu, Sigma, size=n)

In [ ]:
pd.DataFrame(xfull).head()

In [ ]:
# Complete data scatter plot
sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1])

Pour générer des valeurs manquantes MCAR, on utilise le code de l'exercice 1 (question 1).

In [ ]:
p = 0.5
xmiss_mcar = np.copy(xfull)
miss_id_mcar = np.random.uniform(0, 1, size=n) < p
xmiss_mcar[miss_id_mcar, 1] = np.nan
M_mcar = np.isnan(xmiss_mcar)
print("Le pourcentage total de NAs est:", np.sum(M_mcar[:, 1]) / n)

## Question 1: génération de valeurs manquantes MAR et MNAR

Considérons que seulement la seconde variable contient des valeurs manquantes. Proposez un code pour générer des valeurs manquantes de type MAR et MNAR en utilisant la fonction de lien `logit` suivante.
$$\mathrm{logit}(x)=1/(1+e^{-(ax+b)}),$$
avec $a \in \mathbb{R}$ et $b \in \mathbb{R}$.

In [ ]:
def logit(x,coeff,intercept):

  res = 1 / (1 + np.exp(-(coeff * x + intercept)))

  return res

On peut fixer $a=-4$ et $b=0$. À ce stade, on ne cherche pas à contrôler finement le pourcentage de valeurs manquantes que l'on génère.

In [ ]:
a = -4
b = 0

### Solution

Notons $X=(X_{.0} \quad X_{.1})$ le jeu de données, avec $X_{.0}=(x_{10},\dots,x_{n0})^T \in \mathbb{R}^n$ la première variable et $X_{.1}=(x_{11},\dots,x_{n1})^T \in \mathbb{R}^n$ la seconde variable. De même, le masque est $M=(M_{.0} \quad M_{.1})$. Le mécanisme est:


* MAR si $$\mathbb{P}(M_{.1}|X)=\mathrm{logit}(X_{.0}).$$
Dans ce cas, le fait que la seconde variable soit manquante dépend de la première variable qui est observée.
* MNAR si $$\mathbb{P}(M_{.1}|X)=\mathrm{logit}(X_{.1}).$$
Dans ce cas, le fait que la seconde variable soit manquante dépend de ses propres valeurs.

In [ ]:
###Generation of MAR values

xmiss_mar = np.copy(xfull)
proba_mar = logit(xfull[:, 0],a,b)
miss_id_mar = np.random.uniform(0, 1, size=n) < proba_mar
xmiss_mar[miss_id_mar, 1] = np.nan
M_mar = np.isnan(xmiss_mar)
print("Le pourcentage de NA dans la seconde variable est:", np.sum(M_mar[:, 1]) / n)

In [ ]:
###Generation of MNAR values

xmiss_mnar = np.copy(xfull)
proba_mnar = logit(xfull[:, 1], a, b)
miss_id_mnar = np.random.uniform(0, 1, size=n) < proba_mnar
xmiss_mnar[miss_id_mnar, 1] = np.nan
M_mnar = np.isnan(xmiss_mnar)
print("Le pourcentage de NA dans la seconde variable est:", np.sum(M_mnar[:, 1]) / n)

On peut aussi représenter les valeurs manquantes sur un nuage de points. On observe bien que :
* pour MCAR : le manque ne dépend pas des valeurs des données, les valeurs manquantes sont présentes dans tout le nuage de points.
* pour MAR : le manque dépend de l'abscisse, c'est-à-dire de la première variable $X_{.1}$ qui n'est pas manquante.
* dans le cas MNAR : le manque dépend de l'ordonnée, c'est-à-dire de la seconde variable $X_{.2}$ qui est manquante.

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M_mcar[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MCAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M_mar[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M_mnar[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MNAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

## Question 2: calcul du biais de la moyenne empirique

Nous calculons les moyennes empiriques de la seconde variable. Interprétez les résultats suivants.

In [ ]:
empirical_mean = np.mean(xfull[:, 1], axis=0)
empirical_mean_mcar = np.nanmean(xmiss_mcar[:, 1], axis=0)
empirical_mean_mar = np.nanmean(xmiss_mar[:, 1], axis=0)
empirical_mean_mnar = np.nanmean(xmiss_mnar[:, 1], axis=0)

In [ ]:
print("Moyenne empirique sans NA:", f"{empirical_mean:.3f}")
print("Moyenne empirique, cas MCAR:", f"{empirical_mean_mcar:.3f}")
print("Moyenne empirique, cas MAR:", f"{empirical_mean_mar:.3f}")
print("Moyenne empirique, cas MNAR:", f"{empirical_mean_mnar:.3f}")

### Solution

Dans le cas MCAR, il n'y a pas de biais. Théoriquement, on a:
$$\mathbb{E}\left[\frac{1}{n_{\textrm{obs}}}\sum_{i=1}^n (1-M_{i1}) X_{i1}\right]=\mathbb{E}[X_{i1}],$$
où $n_{\textrm{obs}}$ est le nombre de valeurs observées dans $X_{i1}$. En effet:
$\mathbb{E}\left[\frac{1}{n_{\textrm{obs}}}\sum_{i=1}^n (1-M_{i1}) X_{i1}\right]=\frac{n}{n_{\textrm{obs}}}\mathbb{E}[(1-M_{.1})]\mathbb{E}[X_{.1}],$ car $M_{.1}$ et $X_{.1}$ sont indépendants dans le cas MCAR. Enfin, on a $\mathbb{E}[(1-M_{.1})]=n_{\textrm{obs}}/n$, $M_{.1}$ étant tirée selon une loi de Bernoulli de paramètre $p=(n-n_{\textrm{obs}})/n$.

La moyenne empirique calculée sur les valeurs observées est biaisée, même dans le cas MCAR. Nous l'avons étudié dans l'exercice 1. Dans le cas MAR, et d'autant plus dans le cas MNAR, le biais de la moyenne empirique est encore plus important que dans le cas MCAR.

Si on regarde les nuages de points indiquant où sont les valeurs manquantes, cette observation était attendue (voir la correction de la question 1).
Dans le cas MNAR, la plupart des valeurs de $X_{.1}$ négatives sont manquantes. La moyenne empirique est ainsi positive. Dans le cas MAR, même si le manque ne dépend pas de la valeur de la variable $X_{.1}$ elle-même mais dépend de $X_{.0}$, la relation linéaire entre les deux variables implique également que beaucoup de valeurs de $X_{.1}$ négatives sont manquantes, et la moyenne empirique est donc positive.


## Question 3: calcul du biais de l'estimateur du maximum de vraisemblance

La question précédente permettait de calculer la moyenne empirique sur les valeurs observées. On va maintenant calculer l'estimateur du maximum de vraisemblance. On reverra en détail dans le TP du module 3 comment obtenir son expression. Cette dernière dépend de la moyenne empirique de $X_{.0}$; au lieu de n'utiliser que les valeurs observées de $X_{.1}$ (comme la moyenne empirique calculée en question 2), il utilise ainsi toutes les valeurs disponibles du jeu de données, et donc le lien entre les variables. Cela permet de mieux préserver la distribution empirique des données.

Nous allons observer que cet estimateur basé sur la vraisemblance permet d'obtenir des résultats non biaisés dans le cas MCAR ou MAR mais biaisé dans le cas MNAR.

Interprétez les résultats suivants. Proposez ensuite un code pour obtenir les résultats sur plusieurs simulations.

In [ ]:
sum(xmiss_mcar[:,0])

In [ ]:
def maximum_likelihood_estimate(miss_id,xmiss):

  mu0 = np.mean(xmiss[:, 0])

  bar_x0 = np.mean(xmiss[~miss_id, 0])
  bar_x1 = np.mean(xmiss[~miss_id, 1])
  sig_0 = np.mean((xmiss[~miss_id, 0] - bar_x0) ** 2)
  sig_01 = np.mean((xmiss[~miss_id, 0] - bar_x0) * (xmiss[~miss_id, 1] - bar_x1))
  mu1 = np.mean(xmiss[~miss_id, 1]) + sig_01 / sig_0 * (mu0 - np.mean(xmiss[~miss_id, 0]))

  return(mu1)

In [ ]:
mle_mcar = maximum_likelihood_estimate(miss_id_mcar,xmiss_mcar)
mle_mar = maximum_likelihood_estimate(miss_id_mar,xmiss_mar)
mle_mnar = maximum_likelihood_estimate(miss_id_mnar,xmiss_mnar)

print("Moyenne empirique sans NA:", f"{empirical_mean:.3f}")
print("Estimateur du maximum de vraisemblance, cas MCAR:", f"{mle_mcar:.3f}")
print("Estimateur du maximum de vraisemblance, cas MAR:", f"{mle_mar:.3f}")
print("Estimateur du maximum de vraisemblance, cas MNAR:", f"{mle_mnar:.3f}")

### Solution

Dans le calcul de l'estimateur du maximum de vraisemblance, le mécanisme de données manquantes n'a pas été pris en compte. C'est pour cette raison que les résultats sont biaisés dans le cas MNAR.

On peut reproduire l'expérience sur plusieurs simulations, et afficher les boxplots des résultats.

In [ ]:
def compute_bias_mle(n_sim, p, a, b, xfull, Mu):
    
    vec_norm2_bias_mcar = []
    vec_norm2_bias_mar = []
    vec_norm2_bias_mnar = []

    n = xfull.shape[0]
    d = xfull.shape[1]

    empirical_mean = np.mean(xfull[:, 1])
    bias = empirical_mean - Mu[1]
    norm2_bias = (bias ** 2)

    for it in range(n_sim):

        ### Generation of missing values
        xmiss_mcar = np.copy(xfull)
        miss_id_mcar = np.random.uniform(0, 1, size=n) < p
        xmiss_mcar[miss_id_mcar, 1] = np.nan

        xmiss_mar = np.copy(xfull)
        proba_mar = logit(xfull[:, 0], a, b)
        miss_id_mar = np.random.uniform(0, 1, size=n) < proba_mar
        xmiss_mar[miss_id_mar, 1] = np.nan

        xmiss_mnar = np.copy(xfull)
        proba_mnar = logit(xfull[:, 1], a, b)
        miss_id_mnar = np.random.uniform(0, 1, size=n) < proba_mnar
        xmiss_mnar[miss_id_mnar, 1] = np.nan

        mle_mcar = maximum_likelihood_estimate(miss_id_mcar, xmiss_mcar)
        mle_mar = maximum_likelihood_estimate(miss_id_mar, xmiss_mar)
        mle_mnar = maximum_likelihood_estimate(miss_id_mnar, xmiss_mnar)

        bias_mcar = mle_mcar - Mu
        bias_mar = mle_mar - Mu
        bias_mnar = mle_mnar - Mu

        norm2_bias_mcar = (bias_mcar ** 2).sum()
        norm2_bias_mar = (bias_mar ** 2).sum()
        norm2_bias_mnar = (bias_mnar ** 2).sum()
        
        vec_norm2_bias_mcar.append(norm2_bias_mcar)
        vec_norm2_bias_mar.append(norm2_bias_mar)
        vec_norm2_bias_mnar.append(norm2_bias_mnar)

    return(norm2_bias, vec_norm2_bias_mcar, vec_norm2_bias_mar, vec_norm2_bias_mnar)

In [ ]:
norm2_bias, vec_norm2_bias_mcar, vec_norm2_bias_mar, vec_norm2_bias_mnar = compute_bias_mle(n_sim=10, p=0.5, a=-4, b=0, xfull=xfull, Mu=Mu)

In [ ]:
res_na = pd.DataFrame({"MCAR":vec_norm2_bias_mcar, "MAR":vec_norm2_bias_mar, "MNAR":vec_norm2_bias_mnar})
ax = sns.boxplot(res_na)
ax.set_title("Biais de l'estimation de la moyenne")
;

# Exercice 3 : difficultés liées à la génération de valeurs manquantes

Dans cet exercice, vous allez appréhender les difficultés que l'on peut rencontrer lorsque l'on cherche à générer des valeurs manquantes.

Considérons le jeu de données Gaussien des exercices précédents avec $d=3$ variables.

In [ ]:
n = 1000
d = 3
Mu = np.repeat(0, d)
Sigma = 0.5 * (np.ones((d, d)) + np.eye(d))

xfull = np.random.multivariate_normal(Mu, Sigma, size=n) #complete dataset

In [ ]:
pd.DataFrame(xfull).head()

## Question 1 : pourcentage de valeurs manquantes

Pour générer des valeurs manquantes de type MCAR, il est simple d'obtenir un certain pourcentage total de valeurs manquantes. On a vu la manière de procéder dans les exercices précédents, en tirant le masque selon une loi de Bernoulli de paramètre $p$ (la probabilité d'être manquant). Si on veut 40% valeurs manquantes au total, on peut choisir $p=0.4$.

Lorsque le but est de générer des valeurs manquantes de type MAR ou MNAR, c'est plus compliqué.

Plus précisément, supposons que l'objectif est de générer des valeurs manquantes MAR dans la deuxième variable avec la fonction logistique, de telle sorte à avoir un mécanisme ci-dessous: $$\mathbb{P}(M_{.1}|X=(X_{.0},X_{.1},X_{.2}))=1/(1+e^{-(a_0X_{.0}+a_2X_{.2}+b)}),$$
avec $a_0 \in \mathbb{R},\ a_2 \in \mathbb{R},\ b \in \mathbb{R}$.

Pour contrôler le pourcentage de valeurs manquantes dans la variable $X_{.1}$, une méthode consiste à choisir aléatoirement les coefficients $a_0$ et $a_2$, puis d'ajuster le choix de $b$.


Comment le choix de l'intercept $b$ est-il ajusté si la fonction suivante `choose_intercept` est utilisée ? Générez des valeurs manquantes MNAR en l'utilisant.

In [ ]:
def logit(x, coeff, intercept):

  res = 1 / (1+np.exp(-(x.dot(coeff) + intercept)))

  return res

In [ ]:
def choose_intercept(xfull, coeff, idx_var, p):
    
    def f(x):
        return logit(xfull[:, idx_var], coeff, x).mean().item() - p
    
    intercepts = optimize.bisect(f, -50, 50)
    
    return intercepts

In [ ]:
idx_var = [0, 2]
coeff = np.random.normal(size=len(idx_var))
intercept = choose_intercept(xfull, coeff, idx_var, p=0.4)

In [ ]:
print("Les coefficients choisis sont:", coeff)
print("L'intercept choisi est:", intercept)

In [ ]:
###Generation of MAR values

xmiss_mar = np.copy(xfull)
proba_mar = logit(xfull[:, idx_var], coeff, intercept)
miss_id_mar = np.random.uniform(0, 1, size=n) < proba_mar
xmiss_mar[miss_id_mar, 1] = np.nan
M_mar = np.isnan(xmiss_mar)
print("Le pourcentage de NA dans la seconde variable est:", np.sum(M_mar[:, 1]) / n)

### Solution

On choisit $b$ tel que, en moyenne, la probabilité d'être manquante pour une valeur de la variable $X_{.1}$ soit égale à $p$. C'est donc un problème d'optimisation, on cherche le zéro de la fonction ci-dessous:

$f(x)=\frac{1}{n}\sum_{i=1}^n 1/(1+e^{-(a_0X_{i0}+a_2X_{i2}+b)})-p$

Pour générer des valeurs manquantes MNAR, on peut considérer le mécanisme ci-dessous:
$$\mathbb{P}(M_{.1}|X)=\mathrm{logit}(X_{.1}).$$
Voici le code.

In [ ]:
### MNAR case
idx_var = [1]
coeff = np.random.normal(size=len(idx_var))
intercept = choose_intercept(xfull, coeff, idx_var, p=0.4)

xmiss_mnar = np.copy(xfull)
proba_mnar = logit(xfull[:, idx_var], coeff, intercept)
miss_id_mnar = np.random.uniform(0, 1, size=n) < proba_mnar
xmiss_mnar[miss_id_mnar, 1] = np.nan
M_mnar = np.isnan(xmiss_mnar)
print("Le pourcentage de NA dans la seconde variable est:", np.sum(M_mnar[:, 1]) / n)

On peut rapidement vérifier, à l'aide de graphiques, la génération des valeurs manquantes.

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M_mar[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M_mnar[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MNAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

## Question 2 : spécificité du cas MAR

La spécificité du cas MAR est que le manque dépend de valeurs observées des données.

Dans le cas MNAR, on peut générer des valeurs manquantes dans chaque variable, par exemple en utilisant la fonction logistique comme ceci:
$$ \forall j \in \{1,\dots,d\}, \mathbb{P}(M_{.j}|X)=1/(1+e^{-(aX_{.j}+b)}).$$

Dans le cas MAR, faut-il considérer certaines variables totalement observées ?

### Solution

La grande majorité des codes pour générer les valeurs manquantes considèrent une ou plusieurs variables totalement observées. Ce n'est pourtant pas nécessaire pour pouvoir simuler des valeurs MAR.

La définition initiale du mécanisme MAR considère des quantités vectorisés dans $\mathbb{P}(M|X_{\mathrm{obs}(M)})$, c'est-à-dire que $M$ est un vecteur de taille $n\times d$ et $X_{\mathrm{obs}(M)}$ est un vecteur de taille le nombre de valeurs observées dans $X$.

En fait, cette représentation vectorisée revient à simuler les valeurs manquantes par ligne (ou par motifs de manque ou *pattern*). Avec trois variables, on peut très bien avoir des valeurs manquantes MAR dans toutes les variables, par exemple avec les motifs de manque suivants et les mécanismes suivants.

Motifs de manque:
- $i \in \mathrm{Pattern}_1$ si $M_{i.}=(1,0,0)$, c'est-à-dire seule la première variable est manquante.
- $i \in \mathrm{Pattern}_2$ si $M_{i.}=(0,1,0)$, c'est-à-dire seule la deuxième variable est manquante.
- $i \in \mathrm{Pattern}_3$ si $M_{i.}=(0,0,1)$, c'est-à-dire seule la troisième variable est manquante.

Mécanismes:
- $i \in \mathrm{Pattern}_1, \mathbb{P}(M_{i1}|X)=\mathrm{logit}(X_{i2},X_{i3})$,
- $i \in \mathrm{Pattern}_2, \mathbb{P}(M_{i2}|X)=\mathrm{logit}(X_{i1},X_{i2})$,
- $i \in \mathrm{Pattern}_3, \mathbb{P}(M_{i3}|X)=\mathrm{logit}(X_{i1},X_{i2})$.

## Question 3 : utilisation de la librairie `pyampute`

Pour générer des valeurs manquantes par motifs de manque, on peut utiliser la librairie `pyampute`, une documentation est disponible [ici](https://rianneschouten.github.io/pyampute/build/html/index.html). La fonction `MultivariateAmputation` permet de générer des valeurs manquantes dans un jeu de données (initialement complet). Il y a deux arguments principaux:
* `prop`: la proportion de valeurs manquantes par variable,
* `patterns`: une liste de dictionnaires comprenant notamment les entrées suivantes
  * `incomplete_vars`: les indices des variables manquantes
  * `weights`: poids sur les variables qui vont avoir une influence sur le manque
  * `mechanism`: mécanisme de données manquantes
  * `freq`: fréquence du pattern dans le jeu de données amputé

  Chaque dictionnaire correspond à la description d'un pattern.

Utilisez la fonction `MultiviriateAmputation` pour générer des valeurs manquantes MAR comme précisé dans la correction de la question précédente, avec des fréquences de motifs de manque de 10%, 50% et 40% respectivement. Les résultats sont-ils cohérents ?

Attention, la librairie `pyampute` n'est pas maintenue depuis 2022. La fonction de visualisation des motifs de manque (module Python `pyampute.exploration`) renvoie une erreur, le code suivant peut être utilisé à la place.

In [ ]:
def plot_patterns(res):

  #### res is a DataFrame containing all possible missing-data patterns of an incomplete dataset
  #### Example: res = np.unique(M,axis=0), with M the mask

  myred = "#B61A51B3"
  myblue = "#006CC2B3"
  cmap = colors.ListedColormap(['#d1e5f0', '#2166ac'])

  fig, ax = plt.subplots(1)
  ax.imshow(res.astype(bool), aspect="auto", cmap=cmap)


  ax.set_yticks(np.arange(0, len(res.index), 1))
  ax.set_yticks(np.arange(-0.5, len(res.index), 1), minor=True)
  ax.set_xticks(np.arange(0, len(res.columns), 1))
  ax.set_xticks(np.arange(-0.5, len(res.columns), 1), minor=True)


  ax.set_xticklabels([k for k in res.columns])
  ax.set_yticklabels([k for k in res.index])
  ax.grid(which="minor", color="w", linewidth=1)
  plt.show()

### Solution

In [ ]:
pattern1 = {"incomplete_vars": [0], "mechanism": "MAR", "freq":0.1}
pattern2 = {"incomplete_vars": [1], "mechanism": "MAR", "freq":0.5}
pattern3 = {"incomplete_vars": [2], "mechanism": "MAR", "freq":0.4}
patterns = [pattern1, pattern2, pattern3]


ma = pyampute.ampute.MultivariateAmputation(prop=0.9, patterns=patterns)
xmiss = ma.fit_transform(xfull)
M = np.isnan(xmiss)
print("Le pourcentage total de NAs est:", np.sum(M)/(n*d))

L'argument `prop` est bien la proportion de lignes contenant au moins une valeur manquante.

In [ ]:
x_cc = pd.DataFrame(xmiss).dropna()
print("Le pourcentage de lignes incomplètes est:", x_cc.shape[0] / n * 100, "%.")

On peut vérifier que la fréquence des motifs de manque a bien été respectée.

In [ ]:
### Visualisation of the missing-data patterns

which_patterns, counts_patterns = np.unique(M, axis=0, return_counts=True)

res = pd.DataFrame(which_patterns * 1, columns=["X1", "X2", "X3"], index=["Complete row", "Pattern1", "Pattern2", "Pattern3"])

In [ ]:
plot_patterns(res)

In [ ]:
### Frequency of the missing-data patterns

res["Percentage"] = counts_patterns / n * 100
res

# Exercice 4 : mécanismes pseudo-réalistes dans un jeu de données réel

Dans cet exercice, vous considérez un jeu de données réel *Breast Cancer Wisconsin* disponible sur la base UCI [ici](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic), qui ne contient initalement aucune valeur manquante. Les variables sont calculées à partir d'image de tumeurs du sein. Plus précisément, elles décrivent chaque noyau cellulaire avec dix mesures (rayon, texture, périmètre, surface, ...). Finalement, les 30 variables disponibles dans le jeu de données correspondent à la moyenne des mesures sur les noyaux, à l'erreur quadratique et à la pire mesure (au sens de la plus grande, et donc celle qui est la plus susceptible d'impliquer un diagnostic de tumeur maligne). Généralement, ce jeu de données est utilisé dans un objectif de prédiction pour classifier les patientes selon le type de la tumeur : maligne ou bénigne.

Le but de cet exercice est de générer des valeurs manquantes avec un motif de manque *pseudo-réaliste*.


In [ ]:
data = load_breast_cancer()
xfull = data['data']  # covariates, without missing values
diagnosis = data['target']  # target variable to predict, when the learning task is prediction
features_names = data['feature_names']

In [ ]:
pd.DataFrame(xfull, columns=features_names).head()

In [ ]:
features_names

In [ ]:
n, d = xfull.shape

## Question 1 : génération d'un masque MCAR basique

Générez des valeurs manquantes MCAR sur toutes les variables, avec une probabilité d'être manquant de $p=0.3$ pour les 10 premières (qui correspondent au moyenne des mesures), de $p=0.6$ pour les 10 suivantes (erreur quadratique) et de $p=0.8$ pour les 10 dernières (pire mesure). Expliquez en quoi ce mécanisme est bien MCAR.

Ce n'est pas le scénario de manque le plus réaliste. On peut imaginer que les valeurs sont calculées à la main par trois personnes différentes et qu'en fonction de leur rigeur et leur temps (indépendants des valeurs des données, on a plus ou moins de valeurs manquantes).

### Solution

Le mécanisme est bien MCAR ici, car la probabilité d'être manquante pour une valeur ne dépend pas des valeurs des données. Le fait d'avoir des probabilités d'être manquant différents selon les variables n'est pas orthogonal au mécanisme MCAR.

In [ ]:
p = [0.3, 0.6, 0.8]
xmiss = np.copy(xfull)
for j in range(10):
  miss_id = np.random.uniform(0, 1, size=n) < p[0]
  xmiss[miss_id, j] = np.nan
for j in range(10, 20):
  miss_id = np.random.uniform(0, 1, size=n) < p[1]
  xmiss[miss_id, j] = np.nan
for j in range(20, 30):
  miss_id = np.random.uniform(0, 1, size=n) < p[2]
  xmiss[miss_id, j] = np.nan

In [ ]:
pd.DataFrame(xmiss,columns=features_names).head()

## Question 2 : utilisation de la librairie de visualisation `missingno`

La librairie `missingno` est une librairie Python de visualisation en présence de valeurs manquantes. Une documentation est disponible [ici](https://github.com/ResidentMario/missingno).

Utilisez les fonctions `matrix` et `bar` de la librairie `missingno` pour visualiser le jeu de données `xmiss` amputé.

### Solution

La fonction `matrix` permet d'avoir une visualisation globale des motifs de manque dans `xmiss`. On observe que les 10 premières variables ont plus de valeurs observées (cases noires) que les 10 suivantes, qui ont elles-mêmes plus de valeurs observées que les 10 dernières. Cela était attendu, étant donné la génération des valeurs manquantes en question 1.

In [ ]:
missingno.matrix(pd.DataFrame(xmiss)) #global visualisation of missing-data patterns

La fonction `bar` permet de visualiser le nombre de valeurs observées par variable (en haut), et le pourcentage de valeurs observées par variable (en ordonnée gauche).

Les résultats restent cohérent avec la génération des valeurs manquantes de la question 1.

In [ ]:
missingno.bar(pd.DataFrame(xmiss))
#percentage of observed values, and number of observed values per variable

## Question 3 : génération d'un masque avec dépendance

Considérons maintenant que les 10 premières variables sont manquantes avec probabilité $p=0.3$. Supposons de plus que si la première variable est manquante, alors la 11ième l'est aussi ; si la seconde variable est manquante, alors la 12ième l'est aussi, et ainsi de suite. En fait, si on reprend l'exemple de la question 1 où les valeurs ont été calculées à la main, on peut supposer qu'il n'y avait que deux personnes. La première personne a soit reporté les valeurs de la moyenne (variables 1 à 10) et l'erreur quadratique (variables 11 à 20), soit aucune des deux valeurs. La seconde personne a quant à elle reporté toutes les valeurs (variables 21 à 30).

Générez le masque correspondant à ce scénario. Utilisez la fonction `heatmap` de `missingno` pour visualiser l'influence de la présence des dix premières variables sur la présence des dix suivantes. Est-ce que le mécanisme reste MCAR ?

### Solution

On observe avec la fonction `heatmap` que la présence des 10 premières variables est directement liée (avec corrélation de 1) à la présence des 10 suivantes, comme attendu avec la génération du masque.

Le mécanisme de données manquantes reste MCAR. La probabilité d'être manquante pour chaque valeur ne dépendant pas des valeurs des données. Ici, nous avons introduit une dépendance entre les masques $M_{.,j}$ et $M_{.,j+10})$ pour $j=1,\dots,10$, mais le masque reste indépendant des valeurs des données, c'est-à-dire $\mathbb{P}(M|X)=\mathbb{P}(M)$.

In [ ]:
p = 0.3
xmiss = np.copy(xfull)
for j in range(10):
  miss_id = np.random.uniform(0, 1, size=n) < p
  xmiss[miss_id, j] = np.nan
  xmiss[miss_id, j+10] = np.nan

In [ ]:
missingno.heatmap(pd.DataFrame(xmiss))

## Question 4 : cas d'un masque dépendant du diagnostic

Un cas qui arrive souvent en pratique est lorsqu'il y a plus ou moins de valeurs manquantes sur un individu en fonction du groupe auquel il appartient. Ici, on peut imaginer qu'il y a plus de valeurs manquantes pour les patientes avec une tumeur bénigne, car les images sont de moins bonne qualité ou proviennent de patientes qui ont une autre pathologie, et que les médecins ne refont donc pas forcément les images pour ces patientes.

Générez les valeurs manquantes dans ce scénario de manque. Quel est le type de mécanisme de données manquantes, en fonction de si la variable `diagnostic` (indiquant si la tumeur est maligne avec un `0` ou bénigne avec un `1`) est observée ou non ?

### Solution

Le mécanisme est MAR si la variable `diagnostic` est complètement observée, et MNAR si elle est complètement manquante (et donc latente).

In [ ]:
p_benign = 0.7  # probability of being missing for the values of the population with a benign tumor
p_malign = 0.1  # probability of being missing for the values of the population with a malignant tumor

xmiss = np.copy(xfull)
for j in range(d):
  benign_idx = np.where(diagnosis == 1)[0]
  miss_id = np.random.uniform(0, 1, size=benign_idx.shape[0]) < p_benign
  xmiss[benign_idx[miss_id], j] = np.nan

for j in range(d):
  malign_idx = np.where(diagnosis == 0)[0]
  miss_id = np.random.uniform(0, 1, size=malign_idx.shape[0]) < p_malign
  xmiss[malign_idx[miss_id], j] = np.nan

In [ ]:
M = np.isnan(xmiss)
print("Le pourcentage total de NAs est dans la population saine:", np.sum(M[diagnosis == 1, :]) / (sum(diagnosis == 1) * d))
print("Le pourcentage total de NAs est dans la population malade:", np.sum(M[diagnosis == 0, :]) / (sum(diagnosis == 0) * d))

### Remarque: amputation sur un jeu de données incomplet

Ce TP n'aborde pas le cas pratique de l'amputation sur un jeu de données incomplet, qui contient des valeurs manquantes *natives*. Dans ce cas, le problème est que l'on ne peut ni avoir de score de référence, ni comparer des méthodes d'imputation en calculant directement l'erreur commise en imputation. Une solution consiste à introduire de nouvelles valeurs manquantes. Il est alors pertinent de les générer selon la distribution des valeurs manquantes natives. C'est difficile, car cela demande d'estimer la distribution $p(M|X)$, et donc de savoir si le mécanisme est MCAR, MAR ou MNAR. Une première étape est de respecter les mêmes patterns que pour les valeurs manquantes natives. Les nouvelles valeurs manquantes peuvent être introduites sur les lignes complètes du jeu de données si cela est possible, ou alors en complétant des patterns déjà existants.

